#1. 라이브러리 임포트

In [ ]:
! pip install albumentations==0.4.6

In [1]:
import torch,gc
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch import Tensor
from torch.utils.data import DataLoader, Dataset
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection import KeypointRCNN
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
from typing import Tuple, List, Sequence, Callable, Dict

In [2]:
gc.collect()
torch.cuda.empty_cache()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# 2. 코랩 연결 부분

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = '/content/drive/MyDrive/'

Mounted at /content/drive


In [3]:
root_dir = '/home/jngeun/motion_keypoint'

In [4]:
feature_extracting=True
num_classes = 48
learning_rate = 5e-4
batch_size = 8
num_epochs = 50
test_dir = 'data/test_imgs'
train_dir = "data/train_imgs"
train_df_csv = "data/train_df.csv"
test_imgs = os.listdir(os.path.join(root_dir,test_dir))

# 3. 함수 정의 부분

In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [6]:
class KeypointDataset(Dataset):
    def __init__(self, data_dir, label_path, phase=None, transforms=None):
        self.data_dir = data_dir
        self.df = pd.read_csv(label_path)
        self.transforms = transforms
        self.phase= phase
    def __len__(self) -> int:
        return self.df.shape[0]
    
    def __getitem__(self, index) -> Tuple[Tensor, Dict]:
        image_id = self.df.iloc[index, 0]
        labels = np.array([1])
        keypoints = self.df.iloc[index, 1:].values.reshape(-1, 2).astype(np.int64)

        x1, y1 = min(keypoints[:, 0]), min(keypoints[:, 1])
        x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1])
        boxes = np.array([[x1, y1, x2, y2]], dtype=np.int64)

        image = cv2.imread(os.path.join(self.data_dir, image_id), cv2.COLOR_BGR2RGB)

        targets ={
            'image': image,
            'bboxes': boxes,
            'labels': labels,
            'keypoints': keypoints
        }

        if self.transforms is not None:
          targets = self.transforms[self.phase](**targets)
        
        image = targets['image']
        image = image / 255.0

        targets = {
            'labels': torch.as_tensor(targets['labels'], dtype=torch.int64),
            'boxes': torch.as_tensor(targets['bboxes'], dtype=torch.float32),
            'keypoints': torch.as_tensor(
                np.concatenate([targets['keypoints'], np.ones((24, 1))], axis=1)[np.newaxis], dtype=torch.float32
            )
        }

        return image, targets

In [7]:
class TestDataset(Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, data_dir, imgs, phase, transforms=None):
        self.data_dir = data_dir
        self.imgs = imgs
        self.phase = phase
        self.transforms = transforms

    def __getitem__(self, idx):
        filename = self.imgs[idx]
        # Read an image with OpenCV
        img = cv2.imread(os.path.join(self.data_dir, self.imgs[idx]))

        if self.transforms:
            augmented = self.transforms[self.phase](image=img)
            img = augmented['image']

        img = img / 255.0
        return filename, img
    
    def __len__(self):
        return len(self.imgs)
  


In [8]:
def get_model() -> nn.Module:
    backbone = resnet_fpn_backbone('resnet50', pretrained=True)
    roi_pooler = MultiScaleRoIAlign(
        featmap_names=['0', '1', '2', '3'],
        output_size=7,
        sampling_ratio=2
    )

    keypoint_roi_pooler = MultiScaleRoIAlign(
        featmap_names=['0', '1', '2', '3'],
        output_size=14,
        sampling_ratio=2
    )

    model = KeypointRCNN(
        backbone, 
        num_classes=2,
        num_keypoints=24,
        box_roi_pool=roi_pooler,
        keypoint_roi_pool=keypoint_roi_pooler
    )

    return model

In [9]:
def set_parameter_requires_grad(model,feature_extracting):
  if feature_extracting:
    for param in model.backbone.parameters():
      param.requires_grad = False
      # False로 바뀐 부분을 학습 안하겠다.


In [10]:
# augmentation 
A_transforms = {
    'train':
        A.Compose([
            A.Resize(108, 192, always_apply=True),
            # A.Rotate(limit=40,p=0.9),
            # A.OneOf([A.HorizontalFlip(p=1),
            #          A.RandomRotate90(p=1),
            #          A.VerticalFlip(p=1)            
            # ], p=0.5),
            # A.OneOf([A.MotionBlur(p=1),
            #          A.GaussNoise(p=1)                 
            # ], p=0.5),
            #A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ],  bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']),
            keypoint_params=A.KeypointParams(format='xy')),
    
    'val':
        A.Compose([
            A.Resize(108, 192, always_apply=True),
            #A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']),
        keypoint_params=A.KeypointParams(format='xy')),
    
    'test':
        A.Compose([
            A.Resize(108, 192, always_apply=True),
          
          #  A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
}

In [11]:
class MapTrainDataset(Dataset):
    """
    Given a dataset, creates a dataset which applies a mapping function
    to its items (lazily, only when an item is called).

    Note that data is not cloned/copied from the initial dataset.
    """

    def __init__(self, dataset):
        self.dataset = dataset


    def __getitem__(self, index):
      targets ={
            'image': self.dataset[index][0],
            'bboxes': self.dataset[index][1]['boxes'],
            'labels': self.dataset[index][1]['labels'],
            'keypoints': self.dataset[index][1]['keypoints'][0][:,:2]
        }
      targets = A_transforms['train'](**targets)  
      image =targets['image']

      targets = {
            'labels': torch.as_tensor(targets['labels'], dtype=torch.int64),
            'boxes': torch.as_tensor(targets['bboxes'], dtype=torch.float32),
            'keypoints': torch.as_tensor(
                np.concatenate([targets['keypoints'], np.ones((24, 1))], axis=1)[np.newaxis], dtype=torch.float32
            )
        }
      return image, targets
    def __len__(self):
        return len(self.dataset)


In [12]:
class MapValidDataset(Dataset):
    """
    Given a dataset, creates a dataset which applies a mapping function
    to its items (lazily, only when an item is called).

    Note that data is not cloned/copied from the initial dataset.
    """

    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
      targets ={
            'image': self.dataset[index][0],
            'bboxes': self.dataset[index][1]['boxes'],
            'labels': self.dataset[index][1]['labels'],
            'keypoints': self.dataset[index][1]['keypoints'][0][:,:2]
        }
      targets = A_transforms['val'](**targets)  
      image =targets['image']

      targets = {
            'labels': torch.as_tensor(targets['labels'], dtype=torch.int64),
            'boxes': torch.as_tensor(targets['bboxes'], dtype=torch.float32),
            'keypoints': torch.as_tensor(
                np.concatenate([targets['keypoints'], np.ones((24, 1))], axis=1)[np.newaxis], dtype=torch.float32
            )
        }
      return image, targets
    def __len__(self):
        return len(self.dataset)


In [13]:
train_data = KeypointDataset(data_dir = os.path.join(root_dir,train_dir),label_path = os.path.join(root_dir,train_df_csv))
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(0.1 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
tng_data = torch.utils.data.Subset(train_data, train_idx)
val_data = torch.utils.data.Subset(train_data, valid_idx)
tng_data_tf = MapTrainDataset(tng_data)
val_data_tf = MapValidDataset(val_data)
train_loader = DataLoader(tng_data_tf, batch_size=batch_size,collate_fn=collate_fn)
val_loader = DataLoader(val_data_tf, batch_size=batch_size,collate_fn=collate_fn)

In [14]:
test_data = TestDataset(os.path.join(root_dir,test_dir), test_imgs,transforms=A_transforms,  phase='test')
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

# 4. 모델 초기화

In [15]:
model = get_model()
model.cuda()
set_parameter_requires_grad(model,feature_extracting)
criterion = nn.MSELoss()
#optimizer = optim.Adam(model.parameters(),lr=learning_rate)
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=5e-4)
#patience만큼 loss가 향상되지 않으면 learning_rate에 factor을 곱해줌 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.1, patience = 5, verbose=True)


## 4.1 모델 로드

In [16]:
# # 4.1 model load
# model = get_model()
# model.cuda()
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(),lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.1, patience = 5, verbose=True)
# checkpoint = torch.load('/content/drive/MyDrive/data/best_model_11.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

# 5. train / save

In [ ]:
min_loss = 9999
for epoch in range(num_epochs):
    model.train()
    cal_loss = []
    loss_for_validation = []
    loop = tqdm(train_loader)
    for i, (images, targets) in enumerate(loop):
        images = list(image.float().to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        #loss = criterion(model(images), targets)
        losses = model(images,targets)
        loss = sum(loss for loss in losses.values())
        loss.backward()
        cal_loss.append(loss)
        optimizer.step()
        if (i+1) % 10 == 0:
            print(f'| epoch: {epoch} | loss: {loss.item():.4f}')
            print()
    with torch.no_grad():
      for k, (val_images, val_targets) in enumerate(val_loader):
        val_images = list(val_image.float().to(device) for val_image in val_images)
        val_targets = [{k: v.to(device) for k, v in t.items()} for t in val_targets]
        val_losses = model(val_images,val_targets)
        val_loss = sum(loss for loss in val_losses.values())
        loss_for_validation.append(val_loss)
    print(f"epoch : {epoch} | validation loss : {sum(loss_for_validation) / len(loss_for_validation)}")
    mean_loss = sum(cal_loss) / len(cal_loss)
    scheduler.step(mean_loss)
    print(optimizer.param_groups[0]['lr'], mean_loss)
    if mean_loss < min_loss:
      min_loss = mean_loss
      torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss}
              ,os.path.join(root_dir,f'data/best_model_{epoch}.pt'))
## 7 7 6

| epoch: 0 | loss: 9.3207

| epoch: 0 | loss: 9.1400

| epoch: 0 | loss: 9.0076

| epoch: 0 | loss: 8.9172

| epoch: 0 | loss: 8.8977

| epoch: 0 | loss: 8.8654

| epoch: 0 | loss: 8.8692

| epoch: 0 | loss: 8.9052

| epoch: 0 | loss: 8.8476

| epoch: 0 | loss: 8.8825

| epoch: 0 | loss: 8.7356

| epoch: 0 | loss: 8.7425

| epoch: 0 | loss: 8.7695

| epoch: 0 | loss: 8.6974

| epoch: 0 | loss: 8.6753

| epoch: 0 | loss: 8.6852

| epoch: 0 | loss: 8.4483

| epoch: 0 | loss: 8.3454

| epoch: 0 | loss: 8.5348

| epoch: 0 | loss: 8.4937

| epoch: 0 | loss: 8.3472

| epoch: 0 | loss: 8.3146

| epoch: 0 | loss: 8.2174

| epoch: 0 | loss: 8.1643

| epoch: 0 | loss: 8.3398

| epoch: 0 | loss: 7.7697

| epoch: 0 | loss: 7.9861

| epoch: 0 | loss: 8.0889

| epoch: 0 | loss: 8.1170

| epoch: 0 | loss: 7.9418

| epoch: 0 | loss: 7.7052

| epoch: 0 | loss: 8.2534

| epoch: 0 | loss: 8.0785

| epoch: 0 | loss: 8.0962

| epoch: 0 | loss: 8.3698

| epoch: 0 | loss: 7.9220

| epoch: 0 | loss: 8.1412

|

| epoch: 1 | loss: 7.9301

| epoch: 1 | loss: 7.6785

| epoch: 1 | loss: 8.1720

| epoch: 1 | loss: 7.4470

| epoch: 1 | loss: 8.0405

| epoch: 1 | loss: 8.0576

| epoch: 1 | loss: 7.5749

| epoch: 1 | loss: 8.1652

| epoch: 1 | loss: 7.6959

| epoch: 1 | loss: 8.0498

| epoch: 1 | loss: 7.3025

| epoch: 1 | loss: 7.5310

| epoch: 1 | loss: 7.8320

| epoch: 1 | loss: 7.8923

| epoch: 1 | loss: 7.8132

| epoch: 1 | loss: 7.7253

| epoch: 1 | loss: 7.1752

| epoch: 1 | loss: 7.0411

| epoch: 1 | loss: 7.5580

| epoch: 1 | loss: 7.7107

| epoch: 1 | loss: 7.3608

| epoch: 1 | loss: 7.3403

| epoch: 1 | loss: 7.3914

| epoch: 1 | loss: 7.5747

| epoch: 1 | loss: 7.6494

| epoch: 1 | loss: 6.7473

| epoch: 1 | loss: 7.1669

| epoch: 1 | loss: 7.6717

| epoch: 1 | loss: 7.5390

| epoch: 1 | loss: 7.4370

| epoch: 1 | loss: 7.0532

| epoch: 1 | loss: 7.6236

| epoch: 1 | loss: 7.5660

| epoch: 1 | loss: 7.3337

| epoch: 1 | loss: 8.0471

| epoch: 1 | loss: 7.2703

| epoch: 1 | loss: 7.6987

|

| epoch: 2 | loss: 7.3409

| epoch: 2 | loss: 7.3392

| epoch: 2 | loss: 7.4243

| epoch: 2 | loss: 7.0888

| epoch: 2 | loss: 7.3385

| epoch: 2 | loss: 7.5596

| epoch: 2 | loss: 7.0724

| epoch: 2 | loss: 7.6508

| epoch: 2 | loss: 7.2619

| epoch: 2 | loss: 7.3302

| epoch: 2 | loss: 6.7110

| epoch: 2 | loss: 7.0074

| epoch: 2 | loss: 7.2434

| epoch: 2 | loss: 7.1567

| epoch: 2 | loss: 7.3318

| epoch: 2 | loss: 7.2576

| epoch: 2 | loss: 6.6664

| epoch: 2 | loss: 6.6255

| epoch: 2 | loss: 7.2101

| epoch: 2 | loss: 7.0680

| epoch: 2 | loss: 6.8923

| epoch: 2 | loss: 6.8807

| epoch: 2 | loss: 6.9921

| epoch: 2 | loss: 7.0372

| epoch: 2 | loss: 6.9219

| epoch: 2 | loss: 6.6225

| epoch: 2 | loss: 6.7911

| epoch: 2 | loss: 7.1390

| epoch: 2 | loss: 6.8880

| epoch: 2 | loss: 6.8911

| epoch: 2 | loss: 6.6349

| epoch: 2 | loss: 6.8827

| epoch: 2 | loss: 6.9748

| epoch: 2 | loss: 6.8271

| epoch: 2 | loss: 7.5230

| epoch: 2 | loss: 6.8079

| epoch: 2 | loss: 7.3857

|

| epoch: 3 | loss: 6.8878

| epoch: 3 | loss: 6.9574

| epoch: 3 | loss: 6.9092

| epoch: 3 | loss: 6.6478

| epoch: 3 | loss: 6.9883

| epoch: 3 | loss: 7.2497

| epoch: 3 | loss: 6.6460

| epoch: 3 | loss: 7.1887

| epoch: 3 | loss: 6.6913

| epoch: 3 | loss: 6.9269

| epoch: 3 | loss: 6.5435

| epoch: 3 | loss: 6.6816

| epoch: 3 | loss: 6.8010

| epoch: 3 | loss: 7.0613

| epoch: 3 | loss: 6.8882

| epoch: 3 | loss: 6.8115

| epoch: 3 | loss: 6.3149

| epoch: 3 | loss: 6.3119

| epoch: 3 | loss: 6.8542

| epoch: 3 | loss: 6.7488

| epoch: 3 | loss: 6.5328

| epoch: 3 | loss: 6.4638

| epoch: 3 | loss: 6.8001

| epoch: 3 | loss: 6.7377

| epoch: 3 | loss: 6.6123

| epoch: 3 | loss: 6.2487

| epoch: 3 | loss: 6.3455

| epoch: 3 | loss: 6.8303

| epoch: 3 | loss: 6.5119

| epoch: 3 | loss: 6.6432

| epoch: 3 | loss: 6.2239

| epoch: 3 | loss: 6.5258

| epoch: 3 | loss: 6.6826

| epoch: 3 | loss: 6.5440

| epoch: 3 | loss: 7.2973

| epoch: 3 | loss: 6.4710

| epoch: 3 | loss: 6.9380

|

| epoch: 4 | loss: 6.5919

| epoch: 4 | loss: 6.5529

| epoch: 4 | loss: 6.4377

| epoch: 4 | loss: 6.3018

| epoch: 4 | loss: 6.5588

| epoch: 4 | loss: 6.9702

| epoch: 4 | loss: 6.2788

| epoch: 4 | loss: 6.8570

| epoch: 4 | loss: 6.3926

| epoch: 4 | loss: 6.5531

| epoch: 4 | loss: 6.2011

| epoch: 4 | loss: 6.4945

| epoch: 4 | loss: 6.4558

| epoch: 4 | loss: 6.7424

| epoch: 4 | loss: 6.6302

| epoch: 4 | loss: 6.5914

| epoch: 4 | loss: 6.1980

| epoch: 4 | loss: 6.0388

| epoch: 4 | loss: 7.4086

| epoch: 4 | loss: 6.3864

| epoch: 4 | loss: 6.2217

| epoch: 4 | loss: 6.3113

| epoch: 4 | loss: 6.2994

| epoch: 4 | loss: 6.3797

| epoch: 4 | loss: 6.2483

| epoch: 4 | loss: 6.0799

| epoch: 4 | loss: 6.0145

| epoch: 4 | loss: 6.5447

| epoch: 4 | loss: 6.1219

| epoch: 4 | loss: 6.2903

| epoch: 4 | loss: 6.0037

| epoch: 4 | loss: 6.3264

| epoch: 4 | loss: 6.2594

| epoch: 4 | loss: 6.1308

| epoch: 4 | loss: 6.9253

| epoch: 4 | loss: 6.0601

| epoch: 4 | loss: 6.8554

|

| epoch: 5 | loss: 6.5673

| epoch: 5 | loss: 6.2526

| epoch: 5 | loss: 6.2024

| epoch: 5 | loss: 6.1421

| epoch: 5 | loss: 6.2926

| epoch: 5 | loss: 6.4286

| epoch: 5 | loss: 5.9967

| epoch: 5 | loss: 6.4335

| epoch: 5 | loss: 6.1135

| epoch: 5 | loss: 6.2114

| epoch: 5 | loss: 6.0344

| epoch: 5 | loss: 6.0847

| epoch: 5 | loss: 6.1894

| epoch: 5 | loss: 6.4292

| epoch: 5 | loss: 6.3713

| epoch: 5 | loss: 6.3158

| epoch: 5 | loss: 5.8864

| epoch: 5 | loss: 5.8863

| epoch: 5 | loss: 6.4231

| epoch: 5 | loss: 6.1493

| epoch: 5 | loss: 6.0051

| epoch: 5 | loss: 6.0606

| epoch: 5 | loss: 6.2168

| epoch: 5 | loss: 6.2212

| epoch: 5 | loss: 6.0613

| epoch: 5 | loss: 5.8569

| epoch: 5 | loss: 5.8522

| epoch: 5 | loss: 6.3261

| epoch: 5 | loss: 5.9309

| epoch: 5 | loss: 6.1733

| epoch: 5 | loss: 5.6508

| epoch: 5 | loss: 5.9219

| epoch: 5 | loss: 6.0239

| epoch: 5 | loss: 5.8516

| epoch: 5 | loss: 6.7067

| epoch: 5 | loss: 5.8694



# 6. test

In [ ]:
#추론

model.eval()
all_predictions = []
files = []
with torch.no_grad():
  loop = tqdm(test_loader)
  for filenames, inputs in loop:
    inputs = inputs.to(device)
    pred = model(inputs)
    # x means pred[0],pred[1],-----,pred[batch]
    predictions = [x['keypoints'][0][:, :2].reshape(-1).detach().cpu().numpy() for x in pred]
   # predictions = [x['keypoints'][0][:,:2].reshape(-1).detach().cpu().numpy() for x in pred]
    files.extend(filenames)
    for prediction in predictions:
      all_predictions.append(prediction)

In [ ]:
pred

In [ ]:
inputs[3]

In [ ]:
pred[4]

# 7. 파일 저장

In [ ]:
all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    all_predictions[i, [2*j for j in range(num_classes//2)]] /= 192 / 1920
    all_predictions[i, [2*j + 1 for j in range(num_classes//2)]] /= 108 / 1080
df_sub = pd.read_csv(os.path.join(root_dir,'data/sample_submission.csv'))
df = pd.DataFrame(columns=df_sub.columns)
df['image'] = files
df.iloc[:, 1:] = all_predictions
df.head()

In [ ]:
from datetime import datetime
now = datetime.now()
timeday = str(now)[5:10]
df.to_csv(os.path.join(root_dir,f'data/submission_{timeday}.csv'), index=False)

In [ ]:
# train 데이터 확인
train_image , target = dataset.__getitem__(0)
np.array(train_image).shape
target_array = np.array(target['keypoints'][0][:,:2])
plt.imshow(np.array(train_image).transpose(1,2,0))

In [ ]:

#test 확인
filename, test_img = test_data.__getitem__(4)
plt.imshow(np.array(test_img).transpose(1,2,0))
print(filename)

In [ ]:
def draw_keypoints(
    image: np.ndarray,
    keypoints: np.ndarray,
    edges: List[Tuple[int, int]] = None,
    keypoint_names: Dict[int, str] = None, 
    boxes: bool = True,
    dpi: int = 200
) -> None:
    """
    Args:
        image (ndarray): [H, W, C]
        keypoints (ndarray): [N, 3]
        edges (List(Tuple(int, int))): 
    """
    np.random.seed(42)
    colors = {k: tuple(map(int, np.random.randint(0, 255, 3))) for k in range(24)}

    if boxes:
        x1, y1 = min(keypoints[:, 0]), min(keypoints[:, 1])
        x2, y2 = max(keypoints[:, 0]), max(keypoints[:, 1])
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 100, 91), thickness=3)

    for i, keypoint in enumerate(keypoints):
        cv2.circle(
            image, 
            tuple(keypoint), 
            3,(255,0,0), thickness=3, lineType=cv2.FILLED)

        if keypoint_names is not None:
            cv2.putText(
                image, 
                f'{i}: {keypoint_names[i]}', 
                tuple(keypoint), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    if edges is not None:
        for i, edge in enumerate(edges):
            cv2.line(
                image, 
                tuple(keypoints[edge[0]]), 
                tuple(keypoints[edge[1]]),
                colors.get(edge[0]), 3, lineType=cv2.LINE_AA)

    fig, ax = plt.subplots(dpi=dpi)
    ax.imshow(image)
    ax.axis('off')
    plt.show()
keypoint_names = {
    0: 'nose',
    1: 'left_eye',
    2: 'right_eye',
    3: 'left_ear', 
    4: 'right_ear', 
    5: 'left_shoulder', 
    6: 'right_shoulder',
    7: 'left_elbow', 
    8: 'right_elbow',
    9: 'left_wrist', 
    10: 'right_wrist',
    11: 'left_hip', 
    12: 'right_hip',
    13: 'left_knee', 
    14: 'right_knee',
    15: 'left_ankle', 
    16: 'right_ankle',
    17: 'neck', 
    18: 'left_palm', 
    19: 'right_palm', 
    20: 'spine2(back)',
    21: 'spine1(waist)', 
    22: 'left_instep',
    23: 'right_instep'
}

edges = [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10), (9, 18),
    (10, 19), (5, 7), (7, 9), (11, 13), (13, 15), (12, 14),
    (14, 16), (15, 22), (16, 23), (20, 21), (5, 6), (5, 11),
    (6, 12), (11, 12), (17, 20), (20, 21), 
]
#draw_keypoints(np.array(test_img).transpose(1,2,0), keypoints, edges, keypoint_names, boxes=False, dpi=400)

In [ ]:
image = cv2.imread('/content/drive/MyDrive/data/test_imgs/697-3-5-34-Z94_C-0000013.jpg', cv2.COLOR_BGR2RGB)
image = image / 255.0
image = image.transpose(2, 0, 1)
image = [torch.as_tensor(image, dtype=torch.float32)]
#model = get_model()
model.eval()
model.cpu()
preds = model(image)
keypoints = preds[0]['keypoints'].detach().numpy().copy()[0]
image = cv2.imread('/content/drive/MyDrive/data/test_imgs/697-3-5-34-Z94_C-0000013.jpg', cv2.COLOR_BGR2RGB)
keypoints = keypoints[:, :2]

edges = [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10), (9, 18),
    (10, 19), (5, 7), (7, 9), (11, 13), (13, 15), (12, 14),
    (14, 16), (15, 22), (16, 23), (20, 21), (5, 6), (5, 11),
    (6, 12), (11, 12), (17, 20), (20, 21), 
]

draw_keypoints(image, keypoints,boxes=False)

In [ ]:
preds[0]

In [ ]:
from torchvision.models.detection import keypointrcnn_resnet50_fpn
image = cv2.imread('/content/drive/MyDrive/data/train_imgs/001-1-1-01-Z17_A-0000001.jpg', cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (1333, 800))
image = image / 255.0
image = image.transpose(2, 0, 1)
image = [torch.as_tensor(image, dtype=torch.float32)]

model = keypointrcnn_resnet50_fpn(pretrained=True, progress=False)
model.eval()
preds = model(image)

keypoints = preds[0]['keypoints'].detach().numpy().copy()[0]
image = cv2.imread('/content/drive/MyDrive/data/train_imgs/001-1-1-01-Z17_A-0000001.jpg', cv2.COLOR_BGR2RGB)
keypoints[:, 0] *= image.shape[1]/1333
keypoints[:, 1] *= image.shape[0]/800
keypoints = keypoints[:, :2]

edges = [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10),
    (5, 7), (7, 9), (5, 11), (11, 13), (13, 15), (6, 12),
    (12, 14), (14, 16), (5, 6)
]

draw_keypoints(image, keypoints, edges, boxes=False)